In [1]:
from utils import load_data

seq_arr, test_seq_arr, labels, word_index, index_word, vs, embedding_matrix = load_data('word', 'glove')
seq_arr.shape, test_seq_arr.shape, embedding_matrix.shape

Using TensorFlow backend.


((1099063, 30), (56370, 30), (59728, 300))

In [2]:
from utils import f1
from keras import callbacks
from timeit import default_timer as timer
from keras import models, losses, metrics, layers, optimizers
from keras.utils import multi_gpu_model
import tensorflow as tf
import re
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import os
import keras

import numpy as np
import pandas as pd

from tensorflow.python.client import device_lib

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'


def get_available_gpus():
    local_device_protos = device_lib.list_local_devices()
    return [x.name for x in local_device_protos if x.device_type == 'GPU']


get_available_gpus()

['/device:GPU:0', '/device:GPU:1']

In [5]:
import numpy as np
import pandas as pd
from collections import defaultdict
import re

import sys
import os


from keras.preprocessing.text import Tokenizer, text_to_word_sequence
from keras.preprocessing.sequence import pad_sequences
from keras.utils.np_utils import to_categorical

from keras.layers import Embedding
from keras.layers import Dense, Input, Flatten
from keras.layers import Conv1D, MaxPooling1D, Embedding, Dropout, LSTM, GRU, Bidirectional, TimeDistributed
from keras.models import Model

from keras import backend as K
from keras.engine.topology import Layer, InputSpec
from keras import initializers

MAX_SENT_LENGTH = 100
MAX_SENTS = 5
MAX_NB_WORDS = 60000
EMBEDDING_DIM = 300
VALIDATION_SPLIT = 0.4


embedding_layer = Embedding(embedding_matrix.shape[0],
                            EMBEDDING_DIM,
                            weights=[embedding_matrix],
                            input_length=MAX_SENT_LENGTH,
                            trainable=True,
                            mask_zero=True)


class AttLayer(Layer):
    def __init__(self, attention_dim):
        self.init = initializers.get('normal')
        self.supports_masking = True
        self.attention_dim = attention_dim
        super(AttLayer, self).__init__()

    def build(self, input_shape):
        assert len(input_shape) == 3
        self.W = K.variable(self.init((input_shape[-1], self.attention_dim)))
        self.b = K.variable(self.init((self.attention_dim, )))
        self.u = K.variable(self.init((self.attention_dim, 1)))
        self.trainable_weights = [self.W, self.b, self.u]
        super(AttLayer, self).build(input_shape)

    def compute_mask(self, inputs, mask=None):
        return mask

    def call(self, x, mask=None):
        # size of x :[batch_size, sel_len, attention_dim]
        # size of u :[batch_size, attention_dim]
        # uit = tanh(xW+b)
        uit = K.tanh(K.bias_add(K.dot(x, self.W), self.b))
        ait = K.dot(uit, self.u)
        ait = K.squeeze(ait, -1)

        ait = K.exp(ait)

        if mask is not None:
            # Cast the mask to floatX to avoid float64 upcasting in theano
            ait *= K.cast(mask, K.floatx())
        ait /= K.cast(K.sum(ait, axis=1, keepdims=True) + K.epsilon(), K.floatx())
        ait = K.expand_dims(ait)
        weighted_input = x * ait
        output = K.sum(weighted_input, axis=1)

        return output

    def compute_output_shape(self, input_shape):
        return (input_shape[0], input_shape[-1])


In [6]:
sentence_input = Input(shape=(MAX_SENT_LENGTH,), dtype='int32')
embedded_sequences = embedding_layer(sentence_input)
l_lstm = Bidirectional(GRU(100, return_sequences=True))(embedded_sequences)
l_att = AttLayer(100)(l_lstm)
sentEncoder = Model(sentence_input, l_att)
sentEncoder.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 100)               0         
_________________________________________________________________
embedding_2 (Embedding)      (None, 100, 300)          17918400  
_________________________________________________________________
bidirectional_1 (Bidirection (None, 100, 200)          240600    
_________________________________________________________________
att_layer_1 (AttLayer)       (None, 200)               20200     
Total params: 18,179,200
Trainable params: 18,179,200
Non-trainable params: 0
_________________________________________________________________


In [7]:
review_input = Input(shape=(MAX_SENTS, MAX_SENT_LENGTH), dtype='int32')
review_encoder = TimeDistributed(sentEncoder)(review_input)
l_lstm_sent = Bidirectional(GRU(100, return_sequences=True))(review_encoder)
l_att_sent = AttLayer(100)(l_lstm_sent, mask = None)

In [12]:
preds = Dense(1, activation='sigmoid')(l_att_sent)
model = Model(review_input, preds)

model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['acc'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 5, 100)            0         
_________________________________________________________________
time_distributed_1 (TimeDist (None, 5, 200)            18179200  
_________________________________________________________________
bidirectional_2 (Bidirection (None, 5, 200)            180600    
_________________________________________________________________
att_layer_2 (AttLayer)       (None, 200)               20200     
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 201       
Total params: 18,380,201
Trainable params: 18,380,201
Non-trainable params: 0
_________________________________________________________________


In [18]:
import numpy as np

In [30]:
breaks

[['PAD', 'I', 'am', '30', ','],
 ['living', 'at', 'home', 'and', 'have', 'no', 'boyfriend', '.'],
 ['I', 'would', 'love', 'a', 'boyfriend', 'and', 'my', 'own', 'home', '.'],
 ['How', 'can', 'I', 'progress', 'my', 'situation', '?']]

In [39]:
s = seq_arr[10]
s

array([   0,    0,    0,    0,    0,    0,    0,   43,   58,   91,  156,
       2609,  319, 5184,   40,    6, 2725,   40,  347,   40,   16,  295,
       2767,   20,   64,    3,  295,   28, 1653,    2], dtype=int32)

In [52]:
s = seq_arr[100]

max_sen = 3
max_sen_len = 10

data = np.zeros(shape = (1, max_sen, max_sen_len))

clauses = []
track = []
j = 0

for ii, idx in enumerate(s):
    w = index_word[idx]
    track.append(idx)
    if w in punc:
        j += 1
        clauses.append(track)
        track = []
        
data[0, 0:j, :] = pad_sequences(clauses, max_sen_len)

In [55]:
data.astype(int)

array([[[  16,  201,   71, 5545,    5, 5384,   61,   17,   29,    2],
        [   0,    0,    0,    0,    0,    0,    0,    0,    0,    0],
        [   0,    0,    0,    0,    0,    0,    0,    0,    0,    0]]])

In [57]:
punc = ['.', ',', '?', '!', ';', ':']
punc_idx = [word_index[i] for i in punc]
punc_idx

[20, 13, 2, 789, 1193, 202]

In [66]:
punc = ['.', ',', '?', '!', ';', ':']
punc_idx = [word_index[i] for i in punc]

# Number of sequences to try
trial_num = 1000

# Maximum number of sentences
max_sen = 3
# Maximum words per sentence
max_sen_len = 10

# Data is initially all 0s
data = np.zeros((trial_num, max_sen, max_sen_len))

# Iterate through the sequences
for i, s in enumerate(seq_arr[:trial_num]):    
    
    # Clauses is a list of lists
    clauses = []
    # Track is a single list
    track = []
    
    # Number of clauses
    j = 0

    # Iterate through the sequence
    for idx in s:
        if j == max_sen:
            break
        
        # Record the index
        track.append(idx)
        
        # If we find punctuation
        if idx in punc_idx:
            j += 1
            clauses.append(track)
            # Reset the tracker
            track = []

    # Record the found clauses padded to the maximum length
    data[i, 0:j, :] = pad_sequences(clauses, max_sen_len)
    
data.shape

(1000, 3, 10)

In [67]:
from timeit import default_timer as timer

In [69]:
def format_clause_data(sequences,
                max_sen, max_sen_len,
                punc = ['.', ',', '?', '!', ';', ':']):
    """Break data into clauses"""

    punc_idx = [word_index[i] for i in punc]

    # Maximum number of sentences
    max_sen = 3
    # Maximum words per sentence
    max_sen_len = 10

    # Data is initially all 0s
    data = np.zeros((len(sequences), max_sen, max_sen_len))

    start = timer()
    
    # Iterate through the sequences
    for i, s in enumerate(seq_arr):
        # Track progress
        if (i + 1) % 10000 == 0:
            print(f'{100 * i / len(sequences):.2f}% complete.', end = '\r')
        
        # Clauses is a list of lists
        clauses = []
        # Track is a single list
        track = []

        # Number of clauses
        j = 0

        # Iterate through the sequence
        for idx in s:
            # If we have already found enough sentences
            if j == max_sen:
                break

            # Record the index
            track.append(idx)

            # If we find punctuation
            if idx in punc_idx:
                j += 1
                clauses.append(track)
                # Reset the tracker
                track = []

        # Record the found clauses padded to the maximum length
        data[i, 0:j, :] = pad_sequences(clauses, max_sen_len)
    
    print(f'Formatted in {timer() - start:.2f} seconds.')
    print('Final data shape: ', data.shape)
    return data

In [ ]:
data = format_clause_data(seq_arr, max_sen = 3,
                          max_sen_len = 10)

In [62]:
for ii in range(10):
    print(ii)
    for jj in range(20):
        if jj > 10:
            print('breaking')
            break
            

0
breaking
1
breaking
2
breaking
3
breaking
4
breaking
5
breaking
6
breaking
7
breaking
8
breaking
9
breaking


In [37]:
[len(i) for i in seq_dict.values()]

[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]

In [38]:
seq_dict[1]

[array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,   56,   17,   29,   33, 3903,  522,   13,   75,   45,   17,
        3678,   41,    5, 3094,   12,   50, 1934,    2], dtype=int32),
 array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,   56,   17,   29,   33, 3903,  522,   13,   75,   45,   17,
        3678,   41,    5, 3094,   12,   50, 1934,    2], dtype=int32)]

In [9]:
print("model fitting - Hierachical attention network")
model.fit(seq_arr, labels, validation_split = 0.4,
          epochs=10, batch_size=256)

model fitting - Hierachical attention network


ValueError: Error when checking input: expected input_3 to have 3 dimensions, but got array with shape (1099063, 30)

In [11]:
from nltk import tokenize


KeyboardInterrupt: 

In [82]:
from bs4 import BeautifulSoup

MAX_SENT_LENGTH = 100
MAX_SENTS = 5
MAX_NB_WORDS = 60000
EMBEDDING_DIM = 50
VALIDATION_SPLIT = 0.4

def clean_str(string):
    """
    Tokenization/string cleaning for dataset
    Every dataset is lower cased except
    """
    string = re.sub(r"\\", "", string)
    string = re.sub(r"\'", "", string)
    string = re.sub(r"\"", "", string)
    return string.strip().lower()




reviews = []
labels = []

short_texts = texts[:10000]

for i, text in enumerate(short_texts):
    sentences = tokenize.sent_tokenize(text)
    reviews.append(sentences)
    labels.append(df_train.loc[i, 'target'])

tokenizer = Tokenizer(nb_words=MAX_NB_WORDS)
tokenizer.fit_on_texts(short_texts)

data = np.zeros((len(short_texts), MAX_SENTS, MAX_SENT_LENGTH), dtype='int32')

for i, sentences in enumerate(reviews):
    for j, sent in enumerate(sentences):
        if j < MAX_SENTS:
            wordTokens = text_to_word_sequence(sent)
            k = 0
            for _, word in enumerate(wordTokens):
                if k < MAX_SENT_LENGTH and tokenizer.word_index[word] < MAX_NB_WORDS:
                    data[i, j, k] = tokenizer.word_index[word]
                    k = k + 1

word_index = tokenizer.word_index
print('Total %s unique tokens.' % len(word_index))

labels = to_categorical(np.asarray(labels))
print('Shape of data tensor:', data.shape)
print('Shape of label tensor:', labels.shape)

indices = np.arange(data.shape[0])
np.random.shuffle(indices)
data = data[indices]
labels = labels[indices]
nb_validation_samples = int(VALIDATION_SPLIT * data.shape[0])

x_train = data[:-nb_validation_samples]
y_train = labels[:-nb_validation_samples]
x_val = data[-nb_validation_samples:]
y_val = labels[-nb_validation_samples:]

Total 15770 unique tokens.
Shape of data tensor: (10000, 5, 100)
Shape of label tensor: (10000, 2)
Number of positive and negative reviews in traing and validation set


In [68]:
x_train.shape

(20000, 15, 100)

In [66]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /home/wjk68/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [114]:
import numpy as np
import pandas as pd
from collections import defaultdict
import re

from bs4 import BeautifulSoup

import sys
import os


from keras.preprocessing.text import Tokenizer, text_to_word_sequence
from keras.preprocessing.sequence import pad_sequences
from keras.utils.np_utils import to_categorical

from keras.layers import Embedding
from keras.layers import Dense, Input, Flatten
from keras.layers import Conv1D, MaxPooling1D, Embedding, Dropout, LSTM, GRU, Bidirectional, TimeDistributed
from keras.models import Model

from keras import backend as K
from keras.engine.topology import Layer, InputSpec
from keras import initializers

MAX_SENT_LENGTH = 100
MAX_SENTS = 15
MAX_NB_WORDS = 20000
EMBEDDING_DIM = 100
VALIDATION_SPLIT = 0.2


def clean_str(string):
    """
    Tokenization/string cleaning for dataset
    Every dataset is lower cased except
    """
    string = re.sub(r"\\", "", string)
    string = re.sub(r"\'", "", string)
    string = re.sub(r"\"", "", string)
    return string.strip().lower()


data_train = pd.read_csv('/home/wjk68/data/nlp-tutorial/labeledTrainData.tsv', sep='\t')
print(data_train.shape)

from nltk import tokenize

reviews = []
labels = []
texts = []

for idx in range(data_train.review.shape[0]):
    text = BeautifulSoup(data_train.review[idx])
    text = clean_str(text.get_text())
    texts.append(text)
    sentences = tokenize.sent_tokenize(text)
    reviews.append(sentences)

    labels.append(data_train.sentiment[idx])

tokenizer = Tokenizer(nb_words=MAX_NB_WORDS)
tokenizer.fit_on_texts(texts)

data = np.zeros((len(texts), MAX_SENTS, MAX_SENT_LENGTH), dtype='int32')

for i, sentences in enumerate(reviews):
    for j, sent in enumerate(sentences):
        if j < MAX_SENTS:
            wordTokens = text_to_word_sequence(sent)
            k = 0
            for _, word in enumerate(wordTokens):
                if k < MAX_SENT_LENGTH and tokenizer.word_index[word] < MAX_NB_WORDS:
                    data[i, j, k] = tokenizer.word_index[word]
                    k = k + 1

word_index = tokenizer.word_index
print('Total %s unique tokens.' % len(word_index))

labels = to_categorical(np.asarray(labels))
print('Shape of data tensor:', data.shape)
print('Shape of label tensor:', labels.shape)

indices = np.arange(data.shape[0])
np.random.shuffle(indices)
data = data[indices]
labels = labels[indices]
nb_validation_samples = int(VALIDATION_SPLIT * data.shape[0])

x_train = data[:-nb_validation_samples]
y_train = labels[:-nb_validation_samples]
x_val = data[-nb_validation_samples:]
y_val = labels[-nb_validation_samples:]

print('Number of positive and negative reviews in traing and validation set')
print(y_train.sum(axis=0))
print(y_val.sum(axis=0))

(25000, 3)
Total 81501 unique tokens.
Shape of data tensor: (25000, 15, 100)
Shape of label tensor: (25000, 2)
Number of positive and negative reviews in traing and validation set
[10011.  9989.]
[2489. 2511.]


In [115]:
GLOVE_DIR = "/mnt/projects/CSE_MSE_RXF131/hpc-members/wjk68/git"
embeddings_index = {}
f = open(os.path.join(GLOVE_DIR, 'glove.6B.100d.txt'))
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

# print('Total %s word vectors.' % len(embeddings_index))

embedding_matrix = np.random.random((len(word_index) + 1, EMBEDDING_DIM))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector

embedding_layer = Embedding(len(word_index) + 1,
                            EMBEDDING_DIM,
                            weights=[embedding_matrix],
                            input_length=MAX_SENT_LENGTH,
                            trainable=True,
                            mask_zero=True)


class AttLayer(Layer):
    def __init__(self, attention_dim):
        self.init = initializers.get('normal')
        self.supports_masking = True
        self.attention_dim = attention_dim
        super(AttLayer, self).__init__()

    def build(self, input_shape):
        assert len(input_shape) == 3
        self.W = K.variable(self.init((input_shape[-1], self.attention_dim)))
        self.b = K.variable(self.init((self.attention_dim, )))
        self.u = K.variable(self.init((self.attention_dim, 1)))
        self.trainable_weights = [self.W, self.b, self.u]
        super(AttLayer, self).build(input_shape)

    def compute_mask(self, inputs, mask=None):
        return mask

    def call(self, x, mask=None):
        # size of x :[batch_size, sel_len, attention_dim]
        # size of u :[batch_size, attention_dim]
        # uit = tanh(xW+b)
        uit = K.tanh(K.bias_add(K.dot(x, self.W), self.b))
        ait = K.dot(uit, self.u)
        ait = K.squeeze(ait, -1)

        ait = K.exp(ait)

        if mask is not None:
            # Cast the mask to floatX to avoid float64 upcasting in theano
            ait *= K.cast(mask, K.floatx())
        ait /= K.cast(K.sum(ait, axis=1, keepdims=True) + K.epsilon(), K.floatx())
        ait = K.expand_dims(ait)
        weighted_input = x * ait
        output = K.sum(weighted_input, axis=1)

        return output

    def compute_output_shape(self, input_shape):
        return (input_shape[0], input_shape[-1])


ValueError: Dimensions must be equal, but are 15 and 100 for 'att_layer_17/mul' (op: 'Mul') with input shapes: [?,15], [?,15,100].

In [119]:
from keras.preprocessing.text import Tokenizer,  text_to_word_sequence
from keras.engine.topology import Layer
from keras import initializers as initializers, regularizers, constraints
from keras.callbacks import Callback, ModelCheckpoint
from keras.utils.np_utils import to_categorical
from keras.layers import Embedding, Input, Dense, LSTM, GRU, Bidirectional, TimeDistributed
from keras import backend as K
from keras import optimizers
from keras.models import Model
def dot_product(x, kernel):
    """
    Wrapper for dot product operation, in order to be compatible with both
    Theano and Tensorflow
    Args:
        x (): input
        kernel (): weights
    Returns:
    """
    if K.backend() == 'tensorflow':
        return K.squeeze(K.dot(x, K.expand_dims(kernel)), axis=-1)
    else:
        return K.dot(x, kernel)


class AttentionWithContext(Layer):
    """
    Attention operation, with a context/query vector, for temporal data.
    Supports Masking.
    Follows the work of Yang et al. [https://www.cs.cmu.edu/~diyiy/docs/naacl16.pdf]
    "Hierarchical Attention Networks for Document Classification"
    by using a context vector to assist the attention
    # Input shape
        3D tensor with shape: `(samples, steps, features)`.
    # Output shape
        2D tensor with shape: `(samples, features)`.
    How to use:
    Just put it on top of an RNN Layer (GRU/LSTM/SimpleRNN) with return_sequences=True.
    The dimensions are inferred based on the output shape of the RNN.
    Note: The layer has been tested with Keras 2.0.6
    Example:
        model.add(LSTM(64, return_sequences=True))
        model.add(AttentionWithContext())
        # next add a Dense layer (for classification/regression) or whatever...
    """

    def __init__(self,
                 W_regularizer=None, u_regularizer=None, b_regularizer=None,
                 W_constraint=None, u_constraint=None, b_constraint=None,
                 bias=True, **kwargs):

        self.supports_masking = True
        self.init = initializers.get('glorot_uniform')

        self.W_regularizer = regularizers.get(W_regularizer)
        self.u_regularizer = regularizers.get(u_regularizer)
        self.b_regularizer = regularizers.get(b_regularizer)

        self.W_constraint = constraints.get(W_constraint)
        self.u_constraint = constraints.get(u_constraint)
        self.b_constraint = constraints.get(b_constraint)

        self.bias = bias
        super(AttentionWithContext, self).__init__(**kwargs)

    def build(self, input_shape):
        assert len(input_shape) == 3

        self.W = self.add_weight((input_shape[-1], input_shape[-1],),
                                 initializer=self.init,
                                 name='{}_W'.format(self.name),
                                 regularizer=self.W_regularizer,
                                 constraint=self.W_constraint)
        if self.bias:
            self.b = self.add_weight((input_shape[-1],),
                                     initializer='zero',
                                     name='{}_b'.format(self.name),
                                     regularizer=self.b_regularizer,
                                     constraint=self.b_constraint)

        self.u = self.add_weight((input_shape[-1],),
                                 initializer=self.init,
                                 name='{}_u'.format(self.name),
                                 regularizer=self.u_regularizer,
                                 constraint=self.u_constraint)

        super(AttentionWithContext, self).build(input_shape)

    def compute_mask(self, input, input_mask=None):
        # do not pass the mask to the next layers
        return None

    def call(self, x, mask=None):
        uit = dot_product(x, self.W)

        if self.bias:
            uit += self.b

        uit = K.tanh(uit)
        ait = dot_product(uit, self.u)

        a = K.exp(ait)

        # apply mask after the exp. will be re-normalized next
        if mask is not None:
            # Cast the mask to floatX to avoid float64 upcasting in theano
            a *= K.cast(mask, K.floatx())

        # in some cases especially in the early stages of training the sum may be almost zero
        # and this results in NaN's. A workaround is to add a very small positive number ε to the sum.
        # a /= K.cast(K.sum(a, axis=1, keepdims=True), K.floatx())
        a /= K.cast(K.sum(a, axis=1, keepdims=True) + K.epsilon(), K.floatx())

        a = K.expand_dims(a)
        weighted_input = x * a
        return K.sum(weighted_input, axis=1)

    def compute_output_shape(self, input_shape):
        return input_shape[0], input_shape[-1]

In [123]:
sentence_input = Input(shape=(MAX_SENT_LENGTH,), dtype='int32')
embedded_sequences = embedding_layer(sentence_input)
l_lstm = Bidirectional(GRU(100, return_sequences=True))(embedded_sequences)
l_att = AttentionWithContext()(l_lstm)
sentEncoder = Model(sentence_input, l_att)

review_input = Input(shape=(MAX_SENTS, MAX_SENT_LENGTH), dtype='int32')
review_encoder = TimeDistributed(sentEncoder)(review_input)
l_lstm_sent = Bidirectional(GRU(100, return_sequences=True))(review_encoder)
l_att_sent = AttentionWithContext()(l_lstm_sent, mask = None)
preds = Dense(2, activation='softmax')(l_att_sent)
model = Model(review_input, preds)

model.compile(loss='categorical_crossentropy',
              optimizer=optimizers.Adam(),
              metrics=['acc', f1])

print("model fitting - Hierachical attention network")
model.fit(data, labels, validation_split = 0.4,
        nb_epoch=10, batch_size=50)

model fitting - Hierachical attention network


/home/wjk68/.local/lib/python3.6/site-packages/ipykernel_launcher.py:20: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.


Train on 15000 samples, validate on 10000 samples
Epoch 1/10
15000/15000 [==============================] - 104s 7ms/step - loss: 0.3680 - acc: 0.8348 - f1: 0.8348 - val_loss: 0.3045 - val_acc: 0.8720 - val_f1: 0.8720
Epoch 2/10
15000/15000 [==============================] - 96s 6ms/step - loss: 0.2419 - acc: 0.9033 - f1: 0.9033 - val_loss: 0.2769 - val_acc: 0.8829 - val_f1: 0.8829
Epoch 3/10
15000/15000 [==============================] - 96s 6ms/step - loss: 0.1610 - acc: 0.9409 - f1: 0.9409 - val_loss: 0.2862 - val_acc: 0.8860 - val_f1: 0.8860
Epoch 4/10
15000/15000 [==============================] - 96s 6ms/step - loss: 0.0941 - acc: 0.9659 - f1: 0.9659 - val_loss: 0.3747 - val_acc: 0.8748 - val_f1: 0.8748
Epoch 5/10
15000/15000 [==============================] - 96s 6ms/step - loss: 0.0488 - acc: 0.9839 - f1: 0.9839 - val_loss: 0.4894 - val_acc: 0.8785 - val_f1: 0.8785
Epoch 6/10
15000/15000 [==============================] - 96s 6ms/step - loss: 0.0201 - acc: 0.9939 - f1: 0.9939 -